## Backend Development

### Introduction to Full Stack

From python package to bigger project: full stack softawre development (APP).
- **Full stack:** backend + frontend
- **Backend:** some functions or algorthims written in python (python modules) (e.g. a set of modules that load dataset, train some algorthm to interpolate dataset, and make prediction)
- **Frontend:** a user interface (in this course based on Next.js, a web-based UI) that links to backend, allowing user to see and interact (e.g. where we can drag a file to pass the dataset for training and making prediction)

We will also have two docker files, one for backend, one for front end; We also have a docker compose file will manage how the two containers built based on the two docker files interact with each other.
- **Docker file for backend:** builds a Python environment that make our Python module run consistently everywhere.
- **Docker file for frontend:** builds the Next.js environment that make our UI work everywhere.
- **Docker Compose file:** Docker Compose manages the backend and frontend containers working together. It defines how the backend container runs, how the frontend container runs, and how the frontend contacts the backend.
- anyone with docker on the computer then should be able to run this app (backend+frontend) regardless to what platform they use.

How backend communicates with frontend:
- Frontend sends a request to backend
- Backend runs Python code, returns a result
- Frontend displays it to the user
- This communication usually happens via REST API (HTTP calls)
- For each function of the backend, there will be a **RESET API** endpoint (a URL)
- The front end can call these end points, send data, and get data returned from these end point (data passed/returned through **json**)
- **HTTP** is the rule of communication between front end and back end (how front end call REST API endpoints)

Model context protocal (MCP):
LLM agents interact with tools, databases, files, APIs, and external systems based on a rule called MCP.

### Backend: FastAPI

- **Fast API:** API means Application Programming Interface, which is just a program that answers requests; we send it a question like “give me item 42,” it sends back an answer in JSON.
- **JSON:** JavaScript Object Notation is a lightweight, text-based format for structuring data as key–value pairs and arrays that’s easy for humans to read and for machines to parse.

In full stack, **FastAPI** is typically the backend: it runs on the server, handles business logic, talks to databases/queues, and is served by an ASGI server like Uvicorn; while the frontend (e.g., Next.js) calls those FastAPI endpoints over HTTP.

**ASGI:** (Asynchronous Server Gateway Interface) is the Python standard that connects web servers to async-capable apps (like FastAPI, which can handle multiple requests in overlapping time).

### Experiment with simple FastAPI app

We have created a conda virtual env api_test for testing FastAPI (dependencies in requirements.txt) and a folder test_fastapi; **Switch to api_test conda virtual env from now**. Next, create a main.py file in the test_api folder with below code:

In [ ]:
# main.py
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

@app.get("/") #defined get endpoint
def hello():
    return {"message": "Hello 👋"}

@app.post("/items") #defined post end point
def create(item: Item):
    return {"ok": True, "item": item}

We can then run the FastAPI app by running below code at test_fastapi folder:

In [ ]:
uvicorn main:app --reload #here we run the app on uvicorn, an ASGI server

It will run on a default port 8000, if we want to change port number:

In [ ]:
# different port
uvicorn main:app --reload --port 3000

Type http://127.0.0.1:8000/docs in the browser, we can see the app is running

We can try click POST /items, sending {"name": 123, "price": "oops"} and see it politely refuse with a helpful error:

Then send {"name":"apple","price":1.2} and it works, the response body is:

Above, we interat with the FastAPI app through browser, we can also interacts using command line URL tool (curl): here we again send a request with correct data shape, and get a response from the app

In [1]:
!curl -X 'POST' \
  'http://127.0.0.1:8000/items' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{"name":"apple","price":1.2}'

{"ok":true,"item":{"name":"apple","price":1.2}}

Break down of above code:

In [ ]:
!curl -X 'POST' #-X option is used to specify the HTTP method, here we use POST method, which is used to send data to a server/app
  'http://127.0.0.1:8000/items' #URL of our FastAPI items endpoint
  -H 'accept: application/json' #sets a request header indicating I want the response in JSON format
  -H 'Content-Type: application/json' #sets a request header indicating I am sending JSON to the app
  -d '{"name":"apple","price":1.2}' #the request body, in JSON format

If we do not want to send data to the app, we can use HTTP get method (only retrieve data from the app):

In [4]:
!curl -X 'GET' \
  'http://127.0.0.1:8000/'

{"message":"Hello 👋"}

Below code generates same result, since GET is curl default

In [5]:
!curl http://127.0.0.1:8000/

{"message":"Hello 👋"}

### Declaration

The first two lines in main.py import the dependencies:
- FastAPI is the web framework (class) we use
- BaseModel (class from Pydantic) is used to define data shapes with type hints and validation

Below code create our app (FastAPI object):

In [ ]:
# then this lines creates our FastAPI app 
app = FastAPI()

We can also create the fastapi object with more metadata declarations

In [ ]:
app = FastAPI(
    title="Fast API APP Test",
    description="app used for testing FastAPI functions",
    version="0.1.0",
    license_info={"name":"MIT"}
)

Then, we defines a Pydantic model named Item, which specifies a specific data shape
- We latter apply this model to the /items POST endpoint
- This means that when we send POSt request to this end point, the datashape of our request body (in JSON) should match the data shape we defined for the Item model

In [ ]:
class Item(BaseModel):
    name: str # name must be string
    price: float # price must be float
# we can also create other Pydantic models to specify data shape for request sent to other endpoints

Next, we define a GET endpoint at root path:

In [ ]:
@app.get("/") #get is the type of the end point, /means the path of the end point is the root path of the API
def hello(): #above @ is a decorator, it tells FastAPI when the user sends a GET request to /, run the function below
    return {"message": "Hello 👋"}

We also define a POST endpoint at /items path:

In [ ]:
@app.post("/items") #post is the type of the end point, /items is the path of the end point of the API
def create(item: Item): #above @ is a decorator, it tells FastAPI when the user sends a POST request to /items, run the function below
    # item: Item tell the API to read the request body as JSON, parse and validate it against the Item model
    # if the body is missing/invalid → return 422 with error details.
    # if valid, we get a Python Item instance (store in item variable) inside the function.
    # and return {“ok”: True, “item”: item}, with status code 200 OK
    return {"ok": True, "item": item}

### Port

- ``http://`` is the protocol (defines how how we talk), here it is HTTP.
- ``127.0.0.1`` is the loopback address, 127.0.0.1 is a localhost, means “this computer”, hence only apps on the same computer can reach it.
- ``:8000`` is the the port number (like a door on the house). Uvicorn’s dev server defaults to 8000 unless you change it.
- ``/`` the root path of the API. ``/items`` is an extra path the API, directing to the POST endpoint named items. 
- So the complete url http://127.0.0.1:8000 = “Use HTTP to talk to the API running on my machine, on port 8000, at the root path.”

### Add a New End Point

In this example we add an GET endpoint named fft, by adding below code to main.py file

In [ ]:
@app.get("/fft")
def fft_endpoint(n: int = 4096, L: float = 2*np.pi, k: int = 0):
    """
    Compute FFT of sin(5x)*cos(9x) on [0,L) with N samples and return the value at harmonic k.
    - n: number of samples
    - L: domain length
    - k: harmonic index (e.g., 0, ±4, ±14). Maps to FFT bin m ≈ k*L/(2π).
    """
    try:
        return fft_at_k(k=k, n=n, L=L)
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
# we defined fft_at_k somewhere else (maybe in the same file, or in another package)

- If we are running a FastAPI app, we first quit it by Control + C
- Then add the above line to main.py and save
- And reload the app.

### Common Endpoint Types

**GET:**
- Read-only, safe, idempotent (repeat times get same result). 
- Use for fetching (reterieve data) or computing without changing server state.
- Typical status codes: 200 OK, 304 Not Modified, 404 Not Found.

**POST:**
- Create or submit data; not idempotent. 
- Use for creating resources, submitting jobs, or sending complex inputs in the body
- Typical codes: 201 Created, 202 Accepted, 400/422 validation errors.

**DELETE:**
- When sending a header to DELETE endpoint, the API usually delete something from the resource.
- Idempotent by convention.
- Typical codes: 200 OK, 202 Accepted, 204 No Content, 404 Not Found.

### Common Status Codes

Check the url: https://researchcomputing.readthedocs.io/en/latest/fast_api.html#Status-codes